In [ ]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
from itertools import compress
from functools import reduce
import operator
import psycopg2
from settings import db_server, db_user, db_password, db_name

In [ ]:
#Reading input file for awards
grammy_file = "Resources/the_grammy_awards.csv"
grammy_data_df = pd.read_csv(grammy_file)
#Filtering records for 2010 to 2019
grammy_accolades_df=grammy_data_df[grammy_data_df['year'].between(2010,2019)]
grammy_accolades_df=grammy_accolades_df.dropna()
grammy_accolades_df=grammy_accolades_df.reset_index(drop=True)
grammy_accolades_df.head()

In [ ]:
#Reading input file for Songs
tsongs_file = "Resources/top_songs.csv"
tsongs_data_df = pd.read_csv(tsongs_file,encoding='cp1252')
#renaming column for further processing
tsongs_data_df=tsongs_data_df.rename(columns={"top genre":"genre"})
#creating dataframe with required columns
tsongs_data_clean_df=tsongs_data_df[['title','artist','genre','year']]
tsongs_data_clean_df.head()

In [ ]:
# creating artist dataframe 
artist=tsongs_data_df.artist.unique()
artist_df=pd.DataFrame(artist)
artist_df['artist_id']=pd.Series(np.arange(1,len(artist_df)+1,1))
artist_df=artist_df.rename(columns={0: "artist"})
artist_df["artist_id"]= artist_df["artist_id"].astype(int)
artist_df=artist_df[['artist_id','artist']]
artist_df=artist_df.rename(columns={'artist': 'artist_name'})
artist_df.head()

In [ ]:
# Creating Genre dataframe
genre=tsongs_data_df.genre.unique()
genre_df=pd.DataFrame(genre)
genre_df['genre_id']=pd.Series(np.arange(1,len(artist_df)+1,1))
genre_df=genre_df.rename(columns={0: "genre"})
genre_df["genre_id"]= genre_df["genre_id"].astype(int)
genre_df=genre_df[['genre_id','genre']]
genre_df.head()

In [ ]:
#Looping to get only the artists who has top songs
index_result=[]
artist_result=[]
song_result=[]
try:
    for songs_data in range(0,len(artist_df['artist_name'])):
        
        artist_accolade_df=grammy_accolades_df['workers'].str.contains(artist_df['artist_name'][songs_data], flags=re.IGNORECASE, regex=True)
        res = list(compress(range(len(artist_accolade_df )), artist_accolade_df))
        #loop to map artist and worker
        for row in range(0,len(res)):
            artist_result.append(artist_df['artist_id'][songs_data])
            index_result.append(res[row])
except:
    print('Not Found')
#reseting index
accolades_songs_df=grammy_accolades_df.iloc[index_result].reset_index(drop=True)
#Adding artist id in the dataset
accolades_songs_df['artist_id']=artist_result
accolades_songs_df["artist_id"]= accolades_songs_df["artist_id"].astype(int)
accolades_songs_df.head()

In [ ]:
tsongs_data_artist_df = pd.merge(tsongs_data_clean_df, artist_df, how='left', left_on='artist',right_on= 'artist_name')
tsongs_df=pd.merge(tsongs_data_artist_df, genre_df, how='left', on=['genre', 'genre'])
tsongs_df["genre_id"]= tsongs_df["genre_id"].astype(int)
tsongs_df["artist_id"]= tsongs_df["artist_id"].astype(int)
tsongs_df=tsongs_df[['artist_id','title','year','genre_id']]
tsongs_df.head()

In [ ]:
#connection and setup postgres database
rds_connection_string = f'{db_user}:{db_password}@{db_server}:5432/{db_name}'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
artist_df.to_sql(name='artist', con=engine, if_exists='append', index=False)
print('Artist data loaded')
artist = pd.read_sql('select * from artist', engine)
artist.head()

In [ ]:
genre_df.to_sql(name='genre', con=engine, if_exists='append', index=False)
print('Genre data loaded')
genre = pd.read_sql('select * from genre', engine)
genre.head()

In [ ]:
tsongs_df.to_sql(name='songs', con=engine, if_exists='append', index=False)
print('Songs data loaded')
songs = pd.read_sql('select * from songs', engine)
songs.head()

In [ ]:
accolades_songs_df.to_sql(name='accolade', con=engine, if_exists='append', index=False)
print('Accolades data loaded')
accolade = pd.read_sql('select * from accolade', engine)
accolade.head()

In [ ]:
engine.dispose()
print('Database connection closed.')